In [1]:
import os
import numpy as np
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt

In [2]:
# path
cur_path = os.getcwd()
file_path = cur_path + "\Documents\\technical-assignment-2020-0823"

rest_data_path = file_path + '\\rest1.csv'

train = pd.read_csv(file_path + '\\rest1.csv', usecols=[1,2])
len(train)

8337945

In [3]:
# 훈련데이터 세트 분할
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train["review"], train["rate"], stratify=train["rate"], test_size=0.01, random_state=1)

In [4]:
# 적은 부분
train = pd.concat([x_test, y_test], axis=1)
len(train)

83380

In [5]:
# 많은 부분
rest = pd.concat([x_train, y_train], axis=1)

len(rest)

8254565

In [8]:
# 많은 부분 저장
file = file_path + "\\rest2.csv"
rest.to_csv(file)

In [9]:
from konlpy.tag import Okt
okt = Okt()

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [10]:
# 토큰화 객체 계속 쓰기
X_train = []
for sentence in train['review']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)
print(len(X_train))

83380


In [ ]:
y_train = train['rate'].to_numpy()
# 토큰화 객체 저장
with open('tokenized_data2.p', 'wb') as file:
    pickle.dump(X_train, file)
    
with open('data2_label.p', 'wb') as file:
    pickle.dump(y_train, file)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

단어 집합(vocabulary)의 크기 : 29441
등장 빈도가 2번 이하인 희귀 단어의 수: 17139
단어 집합에서 희귀 단어의 비율: 58.21473455385346
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.397535448443756
단어 집합의 크기 : 12304


In [13]:
y_train = np.array(train['rate'])

In [14]:
# 빈 샘플 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

83265
83265


C:\Users\yhc\anaconda3\envs\py37\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [15]:
X_train = pad_sequences(X_train, maxlen = 30)